Heaviy adapted from Week5 tutorial

In [15]:

import transformers
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import pandas as pd
import torch

initialisation

In [16]:
#device = torch.cuda.current_device() if torch.cuda.is_available() else "cpu" # This line checks if a GPU is available and sets the device to GPU (e.g., cuda:0) or CPU.
device = torch.device("cpu")
print(device)


# Initialise the model and tokenizer to a pre-trained model. Suggestions: facebook/opt-350m, bigscience/bloom-560m

# model is made to classify sentence into 1/3 categories -> wa is a seq classification task
model = AutoModelForSequenceClassification.from_pretrained("allenai/longformer-base-4096", num_labels=3)
tokenizer = AutoTokenizer.from_pretrained("allenai/longformer-base-4096")
model = model.to(device)

cpu


Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Data processing

In [17]:
from datasets import Dataset

fp = "../DATASETS/work_arrangements_development_set.csv"
df = pd.read_csv(fp)
df.drop("id", axis=1, inplace=True) #get rid of id column
df.rename(columns={"job_ad": "text"}, inplace=True)

#https://github.com/huggingface/notebooks/blob/main/examples/text_classification-tf.ipynb -> credit
id2label = {
    0:'Hybrid',
    1:'OnSite',
    2:'Remote'
            }
label2id = {val: key for key, val in id2label.items()}

df['labels'] = df['y_true'].map(label2id)    #convert label to enum
print(df)



                                                 text  y_true  labels
0   Job title: CEO\nAbstract: Exciting opportunity...  Remote       2
1   Job title: Home-Based Online ESL Teacher (Onli...  Remote       2
2   Job title: Safeguarding, De La Salle\nAbstract...  Hybrid       0
3   Job title: Delivery Driver\nAbstract: Pickup t...  OnSite       1
4   Job title: Store Supervisor\nAbstract: We are ...  OnSite       1
..                                                ...     ...     ...
94  Job title: Senior Pipeline Technical Director\...  Hybrid       0
95  Job title: Customer Support Administrator\nAbs...  OnSite       1
96  Job title: Remote Writing Evaluator for AI (As...  Remote       2
97  Job title: People & Culture Advisor\nAbstract:...  Hybrid       0
98  Job title: Draftsperson\nAbstract: Residential...  Hybrid       0

[99 rows x 3 columns]


tokenisation

In [18]:
#tokenised
data = Dataset.from_pandas(df)

data = data.map(lambda samples: tokenizer(samples["text"], truncation=True, padding="max_length", max_length=512), batched=True)
data.set_format("torch")    #conversion to pyTorch tensors

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

finetuning

In [19]:
import torch

#GPU sanity check

print(torch.__version__)
print("CUDA Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Device:", torch.cuda.get_device_name(0))
    print("Memory allocated:", torch.cuda.memory_allocated())

2.6.0+cu118
CUDA Available: True
Device: NVIDIA GeForce GTX 1650 Ti
Memory allocated: 1807878656


In [ ]:
import os

training_args = transformers.TrainingArguments(
    output_dir="./results",
    num_train_epochs=4,
    per_device_train_batch_size=1,  #keep at 1, otherwise crash
    logging_dir="./logs",
    report_to="none", 
    logging_strategy="steps",
    logging_steps=10,
    logging_first_step=True, 
    disable_tqdm=False, 
)

# === 6. Trainer and training ===
trainer = transformers.Trainer(
    model=model,
    args=training_args,
    train_dataset=data,
    tokenizer=tokenizer
)

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

trainer.train()

/tmp/ipykernel_462/3886819987.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = transformers.Trainer(


Step,Training Loss
1,1.022300
10,1.138300
20,1.128900
30,1.207100
40,1.076000
50,1.112100
60,1.263200
70,1.006800
80,1.046200
90,1.044600


Eval

In [ ]:
#get test data

fp = "../DATASETS/work_arrangements_test_set.csv"
df = pd.read_csv(fp)
df.drop("id", axis=1, inplace=True) #get rid of id column
df.rename(columns={"job_ad": "text"}, inplace=True)

df


data = Dataset.from_pandas(df)
data = data.map(lambda samples: tokenizer(samples["text"], truncation=True, padding="max_length", max_length=512), batched=True)
data.set_format("torch")    #conversion to pyTorch tensors

print(data)


Map:   0%|          | 0/99 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'y_true', 'input_ids', 'attention_mask'],
    num_rows: 99
})


In [ ]:
# Set the model in evaluation mode
model.eval()

correct = 0

print(f"CORRECT: prediction == label")

with torch.no_grad():
    for i in range(len(data)):
        sample = data[i]
        #print(sample)

        input_ids = sample["input_ids"].unsqueeze(0).to(device)
        attention_mask = sample["attention_mask"].unsqueeze(0).to(device)
        outputs = model(input_ids=input_ids,
                        attention_mask=attention_mask)
        
        #print(outputs)
        logits = outputs.logits
        print(logits)
        pred = torch.argmax(logits, dim=1)  #returns tensor
        predicted_label = id2label[pred.item()]
        if predicted_label == sample['y_true']:
            correct+=1
            print(f"CORRECT: {predicted_label} == {sample['y_true']}")
        else:
            print(f"INCORRECT: {predicted_label} == {sample['y_true']}")

print(f"Accuracy = {correct/len(data)}")


CORRECT: prediction == label
tensor([[-0.8617,  1.0222, -0.1750]], device='cuda:0')
CORRECT: OnSite == OnSite
tensor([[-0.8616,  1.0221, -0.1751]], device='cuda:0')
CORRECT: OnSite == OnSite
tensor([[-0.8617,  1.0222, -0.1750]], device='cuda:0')
INCORRECT: OnSite == Remote
tensor([[-0.8617,  1.0222, -0.1750]], device='cuda:0')
INCORRECT: OnSite == Hybrid
tensor([[-0.8616,  1.0221, -0.1751]], device='cuda:0')
INCORRECT: OnSite == Remote
tensor([[-0.8617,  1.0222, -0.1750]], device='cuda:0')
CORRECT: OnSite == OnSite
tensor([[-0.8617,  1.0222, -0.1750]], device='cuda:0')
INCORRECT: OnSite == Hybrid
tensor([[-0.8618,  1.0222, -0.1750]], device='cuda:0')
INCORRECT: OnSite == Remote
tensor([[-0.8618,  1.0222, -0.1750]], device='cuda:0')
INCORRECT: OnSite == Hybrid
tensor([[-0.8617,  1.0222, -0.1750]], device='cuda:0')
INCORRECT: OnSite == Remote
tensor([[-0.8618,  1.0222, -0.1750]], device='cuda:0')
INCORRECT: OnSite == Hybrid
tensor([[-0.8617,  1.0222, -0.1750]], device='cuda:0')
CORRECT: 